In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from scipy import stats

import ajf_plts

SEED = ajf_plts.get_seed("4_generate_vehicle_loads.ipynb")

plt.style.use(["./ajf_plts/base.mplstyle", "./ajf_plts/legend_frame.mplstyle"])

# Load WIM data

In [2]:
wim_df = pd.read_feather("../data/wim_data.feather")
wim_df.head()

,Axles,S0,S1,S2,S3,S4,S5,SiteID,Timestamp,VehicleClass,W1,W2,W3,W4,W5,W6,dayofweek,hour,month
0,5,0.0,5.70,7.0,17.55,18.8,NaN,1015,2014-03-08 16:59:21.999958,5,50013.915,74530.540,110815.145,50013.915,47071.92,0.0,5,16,3
1,2,0.0,5.80,NaN,NaN,NaN,NaN,1015,2014-03-08 17:03:58.000000,4,50994.580,108853.815,0.000,0.000,0.00,0.0,5,17,3
2,2,0.0,6.55,NaN,NaN,NaN,NaN,1015,2014-03-08 17:12:53.000035,4,57859.235,98066.500,0.000,0.000,0.00,0.0,5,17,3
3,2,0.0,4.70,NaN,NaN,NaN,NaN,1015,2014-03-08 17:13:32.000045,4,40207.265,57859.235,0.000,0.000,0.00,0.0,5,17,3
4,2,0.0,6.10,NaN,NaN,NaN,NaN,1015,2014-03-08 17:15:04.999973,4,57859.235,84337.190,0.000,0.000,0.00,0.0,5,17,3


# Load Speed Data

In [3]:
speed_df = pd.read_feather('../data/speed_data.feather')

# Load Vehicle Counts

In [4]:
df = pd.read_parquet('./vehicle_counts.parquet').sort_values(['seq_month', 'day', 'hour', 'minute']).reset_index(drop=True)
df.head()

,seq_month,year,month,day,hour,minute,dayofweek,air_temperature,freq,E_val,AxleCount
0,0,0,0,1,1,36,6,4.880000,5.786878,2.883737e+10,4
1,0,0,0,1,4,43,6,3.555000,5.795990,2.892656e+10,3
2,0,0,0,1,5,10,6,3.216667,5.799653,2.896242e+10,3
3,0,0,0,1,5,12,6,3.200000,5.799898,2.896483e+10,3
4,0,0,0,1,6,57,6,3.655000,5.801216,2.897772e+10,2


# Randomly choose vehicles

In [5]:
RNG = np.random.Generator(np.random.PCG64(SEED))

WEIGHT_COLS = [f'W{i+1}' for i in range(6)]
SPACING_COLS = [f'S{i}' for i in range(6)]

df[WEIGHT_COLS + SPACING_COLS] = np.nan

for item in df.groupby('AxleCount').seq_month.count().iteritems():
    vehicles = wim_df.loc[wim_df.Axles == item[0]].sample(n=item[1], replace=True, random_state=RNG)[WEIGHT_COLS+SPACING_COLS]
    df.loc[df.AxleCount == item[0], WEIGHT_COLS+SPACING_COLS] = vehicles.values
    df.loc[df.AxleCount == item[0], 'speed'] = speed_df.x.sample(n=item[1], weights=speed_df[str(item[0])], replace=True, random_state=RNG).values


# Save data

In [6]:
df.to_parquet('./vehicle_loads.parquet')